In [2]:
#all imports
import carla #the sim library itself
import time # to set a delay after each photo
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random
import glob
import os
try:
    sys.path.append(glob.glob('./carla/dist/carla-0.9.14-py3.7-win-amd64.egg')[0])
except IndexError:
    print('Couldn\'t import Carla egg properly')
import carla
try:
    sys.path.insert(0,r'C:\Users\ashokkumar\source\repos\AD\carla')
except IndexError:
    pass

from carla import ColorConverter as cc
from agents.navigation.global_route_planner import GlobalRoutePlanner

from scipy.spatial import KDTree

SECONDS_PER_EPISODE = 25

SECONDS_PER_EPISODE = 10  # seconds per episode maybe needs to be removed later
FIXED_DELTA_SECONDS = 0.02 # seconds per frame
SHOW_PREVIEW = True # Show preview for debugging
NO_RENDERING = False # No rendering for training
SYNCRONOUST_MODE = False # Synchronous mode for training
SHOW_CAM = SHOW_PREVIEW     # render camera



In [3]:
## INITIALIZE THE SIMULATOR

# connect to the sim 
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
world = client.load_world('Town02')

settings = world.get_settings()
settings.no_rendering_mode = NO_RENDERING
settings.synchronous_mode = SYNCRONOUST_MODE
settings.fixed_delta_seconds = FIXED_DELTA_SECONDS
world.apply_settings(settings)
blueprint_library = world.get_blueprint_library()
actor_list = []
vehicle = None
collision_hist = []

# get map look at the map
if SHOW_CAM:
    spectator = world.get_spectator()

town_map = world.get_map()
roads = town_map.get_topology()


In [4]:

blueprint_library = world.get_blueprint_library()
model_3 = blueprint_library.find('vehicle.tesla.model3')

Current waypoint: Location(x=193.682220, y=172.089539, z=0.000000)
Target waypoint: Location(x=193.681534, y=169.089539, z=0.000000)

In [5]:
spawn_points = world.get_map().get_spawn_points()

for i, spawn_point in enumerate(spawn_points):
    world.debug.draw_string(spawn_point.location, str(i), life_time=100)
    world.debug.draw_arrow(spawn_point.location, spawn_point.location + spawn_point.get_forward_vector(), life_time=100)



In [6]:
vehicle = world.spawn_actor(model_3, spawn_points[14])
traffic_lights = world.get_actors().filter('traffic.traffic_light')


In [7]:
map = world.get_map()
route_planner = GlobalRoutePlanner(map, sampling_resolution=1)
spawn_points = map.get_spawn_points()    

In [8]:
start_waypoint = spawn_points[14]
dest_waypoint = random.choice(spawn_points)
route = route_planner.trace_route(start_waypoint.location, dest_waypoint.location)
print(len(route))
spectator.set_transform(vehicle.get_transform())

207


In [9]:
for wp in route:
    world.debug.draw_string(wp[0].transform.location, 'O', draw_shadow=False,
                            color=carla.Color(r=0, g=255, b=0), life_time=1000,
                            persistent_lines=True)
    world.debug.draw_point(wp[0].transform.location, size=0.1, color=carla.Color(r=0, g=255, b=0), life_time=1000,
                            persistent_lines=True)

In [10]:
traffic_lights = world.get_actors().filter('traffic.traffic_light')


In [11]:
def update_traffic_light_info(traffic_lights, route):
    """
    Updates the information of traffic lights that are within 2 meters
    of any waypoint in the current route.
    """
    traffic_light_info = []
    for tl in traffic_lights:
        tl_waypoints = tl.get_stop_waypoints()

        if tl_waypoints:
            tl_location = tl_waypoints[0].transform.location
            # Check if the traffic light waypoint is within 2 meters of any waypoint in the route
            for waypoint, _ in route:
                if waypoint.transform.location.distance(tl_location) < 2:
                    tl_info = (tl, tl_waypoints[0])
                    if tl_info not in traffic_light_info:
                        traffic_light_info.append(tl_info)
    
    return traffic_light_info


In [12]:
traffic_lights_info = update_traffic_light_info(traffic_lights, route)
for x in traffic_lights_info:
    print(x[0].get_state())
    print(x[1].transform.location)
    world.debug.draw_string(x[1].transform.location, 'O', draw_shadow=False, color=carla.Color(r=0, g=0, b=255), life_time=10, persistent_lines=True)

Red
Location(x=132.130005, y=219.192657, z=0.000000)
Red
Location(x=153.534256, y=187.622604, z=0.000000)
Green
Location(x=189.681595, y=169.374588, z=0.000000)
Red
Location(x=171.956711, y=240.947983, z=0.000000)


In [13]:
print(len(traffic_lights_info))
world.debug.draw_string(start_waypoint.location, 'O', draw_shadow=False, color=carla.Color(r=0, g=0, b=255), life_time=10, persistent_lines=True)

4


In [14]:
def get_traffic_light_info(vehicle_transform, current_waypoint, route, traffic_light_info):
    vehicle_location = vehicle_transform.location
    # Find the segment of the route that is ahead of the vehicle
    route_tl = _get_route_segment(current_waypoint,route, length=51)
    print(len(route_tl))
    # Check for any traffic lights that impact this segment
    for tl, tl_waypoint in traffic_light_info:
        if any(_is_waypoint_close(waypoint, tl_waypoint) for waypoint, _ in route_tl):
            print('found')
            stop_distance = tl_waypoint.transform.location.distance(vehicle_location)
            return tl.get_state(), stop_distance
    return 0, 0  # Default if no traffic light affects the route

def _get_route_segment( current_waypoint, route, length=51):
    """Returns a sublist of self.route starting from the current_waypoint."""
    if not route:
        print('no route')
        return []
    try:
        start_index = next(
            i for i, (waypoint, _) in enumerate(route)
            if waypoint == current_waypoint
        )
        end_index = min(start_index + length, len(route))
        return route[start_index:end_index]
    except StopIteration:
        return []
    except Exception as e:
        return []
    
def _is_waypoint_close( waypoint, target_waypoint, threshold=2.0):
    """Checks if a waypoint is within a certain distance (threshold) of the target waypoint."""
    distance = waypoint.transform.location.distance(target_waypoint.transform.location)
    return distance < threshold

In [15]:
world.tick()
vehicle_transform = vehicle.get_transform()
state, distance = get_traffic_light_info(vehicle_transform, route[0][0], route, traffic_lights_info)
print(state, distance)

51
found
Green 8.807050704956055


In [16]:
current_waypoint = route[0][0]
print(current_waypoint)
print(start_waypoint)

Waypoint(Transform(Location(x=189.681580, y=161.067245, z=0.000000), Rotation(pitch=360.000000, yaw=90.077835, roll=0.000000)))
Transform(Location(x=189.929993, y=160.580002, z=0.500000), Rotation(pitch=0.000000, yaw=89.999954, roll=0.000000))


In [17]:
vehicle_location = vehicle_transform.location
current_waypoint = route[0][0]
# Find the segment of the route that is ahead of the vehicle
route_tl = _get_route_segment(current_waypoint ,route, length=51)
print(len(route_tl))
# Check for any traffic lights that impact this segment
for tl, tl_waypoint in traffic_lights_info:
    if any(_is_waypoint_close(waypoint, tl_waypoint) for waypoint, _ in route_tl):
        print('found')
        stop_distance = tl_waypoint.transform.location.distance(vehicle_location)
        tl.set_state(carla.TrafficLightState.Red)
        state, distance = tl.get_state(), stop_distance
        

print(state, distance)

51
found
Green 8.807050704956055


In [18]:
tl.set_state(carla.TrafficLightState.Red)
world.tick()
print(tl.get_red_time())
print(tl.get_green_time())


2.0
10.0


In [22]:
tl.set_state(carla.TrafficLightState.Green)
tl.get_state()

carla.libcarla.TrafficLightState.Green